# Event-Based Object Tracking & Motion Detection

Detecting and tracking multiple objects in event-based video using contour detection and edge-pixel analysis.

## Method

1. **Thresholding** — Adaptive threshold to extract event pixels
2. **Contour Detection** — Find object boundaries using `cv2.findContours`
3. **Bounding Boxes** — Draw rectangles around detected objects
4. **Motion Direction** — Compare edge pixel sums to determine movement direction

## Direction Logic

| Edge Comparison | Result |
|-----------------|--------|
| `right_sum > left_sum` | Moving Right |
| `bottom_sum > top_sum` | Moving Down |

In [1]:
#importing libraries
import cv2

video = cv2.VideoCapture("ECE2195_HW6.mp4")

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('object_tracking.mp4', fourcc, video.get(cv2.CAP_PROP_FPS), 
                      (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))

while True:
    
    # Read frame from the video
    ret, frame = video.read()
    if not ret:
        break
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply adaptive threshold to separate black and white pixels
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    
    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Loop over each contour
    for contour in contours:
        if 200< cv2.contourArea(contour):
            
            # Get the bounding box coordinates
            x, y, w, h = cv2.boundingRect(contour)

            # Extract the edge pixels using main frame
            top_edge = frame[y, x:x+w]
            bottom_edge = frame[y+h-1, x:x+w]
            left_edge = frame[y:y+h, x]
            right_edge = frame[y:y+h, x+w-1]

            # Compute the sums of the edge pixels
            top_sum = top_edge.sum()
            bottom_sum = bottom_edge.sum()
            left_sum = left_edge.sum()
            right_sum = right_edge.sum()

            #find directions
            if left_sum < right_sum:
                direction = "Right"
            else:
                direction = "Left"

            if top_sum > bottom_sum:
                direction += " Up"
            else:
                direction += " Down"


            # Draw the bounding box on the frame
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Add the edge sum text to the frame
            cv2.putText(frame, direction, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
    
    # Write the frame to the output video file
    out.write(frame)
    
    # Display the frame
    cv2.imshow('Frame', frame)
    

    #slow video down to check for accuracy
#     cv2.waitKey(int(1000 / video.get(cv2.CAP_PROP_FPS) / 50))
    
    # Wait for a key press and exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# Release the video capture and close the window
video.release()
out.release()
cv2.destroyAllWindows()